# # RooTrip 추천 시스템 v2

In [3]:
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 데이터프레임 생성 (예시)
# data = {
#     'id': [1, 2, 3, 4],
#     'Hashtag': [
#         '[{"tag":"어트랙션"},{"tag":"여행"}]',
#         '[{"tag":"바다"},{"tag":"여행"}]',
#         '[{"tag":"산"},{"tag":"바다"},{"tag":"관광"}]',
#         '[{"tag":"부산"},{"tag":"바다"},{"tag":"해운대"}]'
#     ],
#     'title': ['어트랙션 여행', '바다 여행', '산 바다 여행', '부산 바다 해운대']
# }

# RooTrip 추천 시스템 v2
import pandas as pd
import numpy as np
import json
import os

# os.chdir('/Users/woojjam/Desktop/study/machine-learning/Projects/rootrip-recommendation')

# df1 = pd.read_csv('rootrip-board.csv')
df1 = pd.DataFrame(data)
print(df1)
# JSON 데이터를 파이썬 객체로 변환
df1['Hashtag'] = df1['Hashtag'].apply(lambda x: json.loads(x.replace('\'', '\"')))

# 사용자 입력 해시태그
user_hashtags = '부산, 여행, 어트랙션'.split(', ')


# Hashtag 열을 정리하여 'soup' 열 생성
def create_soup(x):
    return ' '.join(tag['tag'] for tag in x)

df1['soup'] = df1['Hashtag'].apply(create_soup)

# CountVectorizer를 사용하여 단어 벡터화
count = CountVectorizer()
count_matrix = count.fit_transform(df1['soup'])

# 코사인 유사도 매트릭스 계산
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# 사용자 입력 해시태그와 각 행의 해시태그 간의 유사도 계산
similarities = cosine_sim[-1]  # 마지막 행은 사용자 입력에 해당

user_hashtag_soup = ' '.join(user_hashtags)
user_hashtag_vector = count.transform([user_hashtag_soup])
similarities = cosine_similarity(user_hashtag_vector, count_matrix)

# 유사도와 영화 제목을 매핑
similarities_with_titles = list(enumerate(similarities[0]))
similarities_with_titles = sorted(similarities_with_titles, key=lambda x: x[1], reverse=True)

print(similarities_with_titles)
# 가장 유사한 영화 추출
recommended_movies = []
for i in range(4):  # 상위 3개 영화 추천
    movie_idx = similarities_with_titles[i][0]
    recommended_movies.append(df1['title'].iloc[movie_idx])

# 사용자에게 추천 목록을 출력
print("추천 영화:")
for movie in recommended_movies:
    print(movie)


   id                                    Hashtag        like      title
0   1              [{"tag":"어트랙션"},{"tag":"여행"}]        1234    어트랙션 여행
1   2                [{"tag":"바다"},{"tag":"여행"}]      124124      바다 여행
2   3    [{"tag":"산"},{"tag":"바다"},{"tag":"관광"}]         211    산 바다 여행
3   4  [{"tag":"부산"},{"tag":"바다"},{"tag":"해운대"}]  1232145123  부산 바다 해운대
[(0, 0.816496580927726), (1, 0.408248290463863), (3, 0.3333333333333334), (2, 0.0)]
추천 영화:
어트랙션 여행
바다 여행
부산 바다 해운대
산 바다 여행


   id                                    Hashtag        like      title
0   1              [{"tag":"어트랙션"},{"tag":"여행"}]        1234    어트랙션 여행
1   2                [{"tag":"바다"},{"tag":"여행"}]      124124      바다 여행
2   3    [{"tag":"산"},{"tag":"바다"},{"tag":"관광"}]         211    산 바다 여행
3   4  [{"tag":"부산"},{"tag":"바다"},{"tag":"해운대"}]  1232145123  부산 바다 해운대
